In [3]:
%matplotlib inline

import os

# Turn off TensorFlow logging messages
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# For reproducibility
os.environ["PYTHONHASHSEED"] = "0"

In [5]:
import pandas as pd


DISPLAY_ALL_TEXT = False

pd.set_option("display.max_colwidth", 0 if DISPLAY_ALL_TEXT else 50)

In [7]:
# Download the spaCy english model
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 2.6 MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047105 sha256=e706187d1a85c041625410d58191efa9b079342f22e845b66cb791e147d4df7b
  Stored in directory: /private/var/folders/16/xhpw7rdx7t1chn5pz7d6km1r0000gn/T/pip-ephem-wheel-cache-gl0kv32r/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [8]:
from utils import load_spam_dataset

df_train, df_test = load_spam_dataset()

# We pull out the label vectors for ease of use later
Y_test = df_test.label.values

FileNotFoundError: [Errno 2] No such file or directory: 'spam'